Our task is to predict flu infections based on the methods described [here](http://www.deltami.edu.pl/temat/matematyka/zastosowania/2019/11/25/Zanim_dopadnie_nas_grypa/). 

We used dataset [Flu cases from 1997 through 2021 by week](https://www.kaggle.com/datasets/chrisleblanc/flu-cases-from-1997-through-2021-by-week?select=flu_cases_1997_2021_raw_data.xlsx).

In [2]:
import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.signal import find_peaks

TODO: zmienić import na ten z ibm cloud

In [99]:
flu_data = pd.read_excel('/content/flu_cases_1997_2021_raw_data.xlsx')
flu_data.head()

,YEAR,WEEK,Total_cases
0,1997,40,0
1,1997,41,11
2,1997,42,17
3,1997,43,8
4,1997,44,10


In [100]:
# we want to obtain date from columns YEAR and WEEK
flu_data['date'] = pd.to_datetime(flu_data.YEAR.astype(str), format='%Y') + \
             pd.to_timedelta(flu_data.WEEK.mul(7).astype(str) + ' days')
flu_data.head()

,YEAR,WEEK,Total_cases,date
0,1997,40,0,1997-10-08
1,1997,41,11,1997-10-15
2,1997,42,17,1997-10-22
3,1997,43,8,1997-10-29
4,1997,44,10,1997-11-05


In [101]:
# we add additional column for day count
days = list(range(0, 7*len(flu_data), 7))
flu_data['days'] = days
flu_data.head()

,YEAR,WEEK,Total_cases,date,days
0,1997,40,0,1997-10-08,0
1,1997,41,11,1997-10-15,7
2,1997,42,17,1997-10-22,14
3,1997,43,8,1997-10-29,21
4,1997,44,10,1997-11-05,28


# Flu cases from 1997 to 2021 - plot

In [56]:
fig = px.line(flu_data, x='date', y="Total_cases", hover_data=['date'])
fig.show()

We can see that data since April 2020 differ from the previous years -- it is probably because of the COVID-19 pandemics. We've decided to delete that data from our dataset.

In [106]:
flu_data = flu_data[~ flu_data['YEAR'].isin([2020, 2021])]

In [107]:
fig = px.line(flu_data, x='date', y="Total_cases", hover_data=['date'])
fig.show()

# Finding local maxima

In [108]:
# peaks - indexes of peaks
peaks, _ = find_peaks(flu_data['Total_cases'], distance=24)

In [126]:
x = flu_data['Total_cases']
fig = px.line(flu_data, x='date', y="Total_cases", hover_data=['date'])
fig.add_scatter(x=flu_data['date'].iloc[peaks], y=x[peaks], mode="markers", marker_symbol='x', marker=dict(size=8), name='local maxima')
fig.show()

In [127]:
# choosing dataframe only for peaks
peaks_flu = flu_data.iloc[peaks]

In [129]:
def PolyCoefficients(x, coeffs):
    o = len(coeffs)
    y = 0
    for i in range(o):
        y += coeffs[i]*x**i
    return y

In [142]:
x = flu_data['days']

In [143]:
p_1 = np.polyfit(x=peaks_flu['days'], y=peaks_flu['Total_cases'], deg=1)
np.polyval(p, list(peaks_flu['days'])[-1] + 365)

3086.3930632635547

In [144]:
p_2 = np.polyfit(x=peaks_flu['days'], y=peaks_flu['Total_cases'], deg=2)
p_3 = np.polyfit(x=peaks_flu['days'], y=peaks_flu['Total_cases'], deg=3)
p_4 = np.polyfit(x=peaks_flu['days'], y=peaks_flu['Total_cases'], deg=4)
p_5 = np.polyfit(x=peaks_flu['days'], y=peaks_flu['Total_cases'], deg=5)
p_6 = np.polyfit(x=peaks_flu['days'], y=peaks_flu['Total_cases'], deg=6)

In [148]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=2)

fig.add_trace(
    go.Scatter(x=x, y=PolyCoefficients(x, p_1), name='deg=1'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=x, y=PolyCoefficients(x, p_2), name='deg=2'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=x, y=PolyCoefficients(x, p_3), name='deg=3'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=x, y=PolyCoefficients(x, p_4), name='deg=4'),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(x=x, y=PolyCoefficients(x, p_5), name='deg=5'),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=x, y=PolyCoefficients(x, p_6), name='deg=6'),
    row=3, col=2
)


fig.update_layout(title_text="Polynomial plots for different degree of polynomial")
fig.show()


# Final model

In [95]:
def KMcK(S1, I1, R1, n, b, a=0.4):
  S = [S1]
  I = [I1]
  R = [R1]
  for i in range(n):
    S.append(S[i] - b * I[i] * S[i])
    I.append(I[i] + b * I[i] * S[i] - a * I[i])
    R.append(R[i] + a * I[i])
  return S, I, R